In [1]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
%matplotlib inline

# 1. Hash function and mining 
A blockchain needs to do two things: gather and order data into blocks, and then chain them together securely using cryptography.
1. Let's start with a simple transaction: Alice sells her car to Bill.
2. The transaction information is recorded and shared with the other computers in the blockchain network.
3. On the network, the record is combined with other transactions into a block—like a traditional computer database. Each transaction is time-stamped. Many blocks are combined into a chain. 
4. Securing the chain: The key to a blockchain's security is something called a hash. It's a bit of cryptographic math that makes the links between blocks virtually unbreakable.A hash function takes the information in each block and uses it to create the hash—a unique string of characters.
5. The hash from one block is added to the data in the next block.So when the next block goes through the hash function, a trace of it is woven into the new hash. And so on, throughout the chain.

* Mining is the process of adding transaction records to blockchain
*  Individual blocks must contain a proof of work to be considered valid. This proof of work is verified by other Bitcoin nodes each time they receive a block. (This is how people make money mining bitcoins :)
* The primary purpose of mining is to allow nodes to reach a secure, tamper-resistant consensus

Mining a block is difficult because the SHA-256 hash of a block's header must be lower than or equal to the target in order for the block to be accepted by the network. This problem can be simplified for explanation purposes: The hash of a block must start with a certain number of zeros(in Bitcoin) or ones or any (in this case ones). The probability of calculating a hash that starts with many ones is very low, therefore many attempts must be made. In order to generate a new hash each round, a nonce is incremented

From http://www.goldmansachs.com/our-thinking/pages/blockchain/
And https://en.bitcoin.it/wiki/Proof_of_work

In [2]:

# Define two functions: one to hash a string and one to mine a nonce for a given string
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

def dumb_hash(message):
    """
    Returns an hexadecimal hash
    """
    return sha256(message)

def mine(message, difficulty=1):
    """
    Given an input string, will return a nonce such that
    hash(string + nonce) starts with `difficulty` ones
    
    Returns: (nonce, niters)
        nonce: The found nonce
        niters: The number of iterations required to find the nonce
    """
    assert difficulty >= 1, "Difficulty of 0 is not possible"
    i = 0
    prefix = '1' * difficulty
    while True:
        nonce = str(i)
        digest = dumb_hash(message + nonce)
        if digest.startswith(prefix):
            return nonce, i
        i += 1

In [3]:
# Test mining nonce of varied difficulty
# Note: Leading ones is 1
nonce, niters = mine('99', difficulty=1)
print('Took %d iterations' % niters)
print('Found Nonce ', nonce)
print(sha256('99' + str(nonce)))

# Note: Leading ones is 7
nonce, niters = mine('99', difficulty=7)
print('Took %d iterations' % niters)
print('Found Nonce ', nonce)
print(sha256('99' + str(nonce)))

Took 67 iterations
Found Nonce  67
10e8d3c45549d1199fdc25c0902659c2c39b1cc0c91178de73d03e40c9cb4936
Took 32370927 iterations
Found Nonce  32370927
1111111e1b2cfb4346775bd7e3c73d3ae541d3b8032907f5eba1af3fb3dcfde3


# 2. A Wallet
In bitcoin a wallet is a private/public key pair. 

The public key is used to receive transactions and the private key is used to spend money. By signing a transaction with our private key, anybody else can verify the signature using our public key.

From: https://nbviewer.jupyter.org/github/julienr/ipynb_playground/blob/master/bitcoin/dumbcoin/dumbcoin.ipynb


In [4]:
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5


class Wallet(object):
    """
    A wallet is a private/public key pair
    """
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random_gen)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)
        
    @property
    def address(self):
        """We take a shortcut and say address is public key"""
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    
    def sign(self, message):
        """
        Sign a message with this wallet
        """
        h = SHA.new(message.encode('utf8'))
        return binascii.hexlify(self._signer.sign(h)).decode('ascii')
    
    
def verify_signature(wallet_address, message, signature):
    """
    verify signature agaisnt message signed with public/private key
    """
    pubkey = RSA.importKey(binascii.unhexlify(wallet_address))
    verifier = PKCS1_v1_5.new(pubkey)
    h = SHA.new(message.encode('utf8'))
    return verifier.verify(h, binascii.unhexlify(signature))


# Check that the wallet signing functionality works
w1 = Wallet()
signature = w1.sign('foobar')
assert verify_signature(w1.address, 'foobar', signature)
assert not verify_signature(w1.address, 'rogue message', signature)       
        
        
        
    

# 3. Doing Transactions
Consists of:
* A Spender: Signs the transaction and has money
* Spender's Wallet. (Address and Public/Private Key)
* Amount of Money and Recipient.
* Transaction fee to incentivise a miner to include the transaction in a block
* GenesisTransaction which is the root of all transactions


In [5]:
class TransactionInput(object):
    """
    An input for a transaction. This points to an output of another transaction
    """
    def __init__(self, transaction, output_index):
        self.transaction = transaction
        self.output_index = output_index
        assert 0 <= self.output_index < len(transaction.outputs)
        
    def to_dict(self):
        d = {
            'transaction': self.transaction.hash(),
            'output_index': self.output_index
        }
        return d
    
    @property
    def parent_output(self):
        return self.transaction.outputs[self.output_index]
    

class TransactionOutput(object):
    """
    An output for a transcation. This specifies an amount and a recipient (wallet)
    """
    def __init__(self, recipient_address, amount):
        self.recipient = recipient_address
        self.amount = amount
        
    def to_dict(self):
        d = {
            'recipient_address': self.recipient,
            'amount': self.amount
        }
        return d

        
def compute_fee(inputs, outputs):
    """
    Compute the transaction fee by computing the difference between total input and total output
    """
    total_in = sum(i.transaction.outputs[i.output_index].amount for i in inputs)
    total_out = sum(o.amount for o in outputs)
    assert total_out <= total_in, "Invalid transaction with out(%f) > in(%f)" % (total_out, total_in)
    return total_in - total_out

    
class Transaction(object):
    def __init__(self, wallet, inputs, outputs):
        """
        Create a transaction spending money from the provided wallet
        """
        self.inputs = inputs
        self.outputs = outputs
        self.fee = compute_fee(inputs, outputs)
        self.signature = wallet.sign(json.dumps(self.to_dict(include_signature=False)))
        
    def to_dict(self, include_signature=True):
        d = {
            "inputs": list(map(TransactionInput.to_dict, self.inputs)),
            "outputs": list(map(TransactionOutput.to_dict, self.outputs)),
            "fee": self.fee
        }
        if include_signature:
            d["signature"] = self.signature
        return d
    
    def hash(self):
        return dumb_hash(json.dumps(self.to_dict()))
    
    
class GenesisTransaction(Transaction):
    """
    This is the first transaction which is a special transaction
    with no input and 25 bitcoins output
    """
    def __init__(self, recipient_address, amount=25):
        self.inputs = []
        self.outputs = [
            TransactionOutput(recipient_address, amount)
        ]
        self.fee = 0
        self.signature = 'genesis'
        
    def to_dict(self, include_signature=False):
        # TODO: Instead, should sign genesis transaction will well-known public key ?
        assert not include_signature, "Cannot include signature of genesis transaction"
        return super().to_dict(include_signature=False)

In [15]:
def compute_balance(wallet_address, transactions):
    """
    Given an address and a list of transactions, computes the wallet balance of the address
    """
    balance = 0
    for t in transactions:
        # Subtract all the money that the address sent out
        for txin in t.inputs:
            if txin.parent_output.recipient == wallet_address:
                balance -= txin.parent_output.amount
        # Add all the money received by the address
        for txout in t.outputs:
            if txout.recipient == wallet_address:
                balance += txout.amount
    return balance        
        
                

In [16]:
# Perform some transactions
alice = Wallet()
bob = Wallet()
charlie = Wallet()

# give alice 25 coins
t1 = GenesisTransaction(alice.address)

# Alice gives bob 5 coins
# Alice gives charlie 5 coins
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 5.0), TransactionOutput(alice.address, 15.0), TransactionOutput(charlie.address, 5.0)]
)

# Bob gives 2 coins to charlie
t3 = Transaction(
    bob,
    [TransactionInput(t2, 2)],
    [TransactionOutput(charlie.address, 2.0)])

# Charlie gives 2 coins to Bob
# Charlier pays a fee of 1 coin
t4 = Transaction(
    charlie,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(bob.address, 2.0), TransactionOutput(charlie.address, 1.0)]
)

transactions = [t1, t2, t3, t4]
print("Alice  has %.02f dumbcoins" % compute_balance(alice.address, transactions))
print("Bob    has %.02f dumbcoins" % compute_balance(bob.address, transactions))
print("Charlie has %.02f dumbcoins" % compute_balance(charlie.address, transactions))

Alice  has 15.00 dumbcoins
Bob    has 2.00 dumbcoins
Charlie has 1.00 dumbcoins


# 4. To Do
* Verify Transactions
* Add Transactions to a block
* Test

In [21]:
# Test if Bob can spend Alice Money

t1 = GenesisTransaction(alice.address)

t2 = Transaction(
    bob,
    [TransactionInput(t1, 0)],
    [TransactionOutput(charlie.address, 10.0)]
)


t3 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(charlie.address, 10.0)]
)

transactions = [t1, t2, t3]

print("Charlie has %.02f dumbcoins" % compute_balance(charlie.address, transactions))

# So apparently, Bob did sent Charlie 10 coins using Alice's money which should not happen

Charlie has 20.00 dumbcoins
